# Day 1 - Calorie Counting

In [41]:

import pandas as pd

with open("./resources/day1.txt") as f:
    input = f.read()

sums = [pd.Series(elve.split("\n")).astype(int).sum() for elve in input.split("\n\n")]
part1 = max(sums)
part2 = pd.Series(sums).sort_values(ascending=False).head(3).sum()
    
part1, part2

(67658, 200158)

# Day 2 - Rock Paper Scissors

In [44]:
import pandas as pd

with open("./resources/day2.txt") as f:
    input = f.read()

mapping = {
    "A": 1,
    "B": 2,
    "C": 3,
    "X": 1,
    "Y": 2,
    "Z": 3,
}

def eval_points(opp: str, me: str):
    if opp == (me-1) or (opp == 3 and me==1): # win 1<2<3
        return me + 6
    elif opp == me: # draw
        return me + 3
    return me # loss

def eval_points2(opp:str, end:str):
    if end==1: # need to loose
        return opp-1 if opp!=1 else 3
    elif end==2: # need to draw
        return opp + 3
    return (opp+1 if opp!=3 else 1) + 6

df = pd.DataFrame([round.split(" ") for round in input.split("\n")], columns=["opp", "me"]).assign(opp=lambda df:df.opp.map(mapping), me=lambda df:df.me.map(mapping))
part1 = df.apply(lambda df:eval_points(df.opp, df.me), axis=1).sum()
part2 = df.apply(lambda df:eval_points2(df.opp, df.me), axis=1).sum()

part1, part2

(15337, 11696)

# Day 3 - Rucksack Reorganization

In [1]:
import pandas as pd

with open("./resources/day3.txt") as f:
    input = f.readlines()

alph = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"

def find_duplicate(left: str, right: str) -> int:
    for c in left:
        if c in right:
            return alph.index(c) + 1

def find_duplicate2(first: str, second: str, third: str) -> int:
    for c in first:
        if c in  second and c in third:
            return alph.index(c) + 1


df = pd.DataFrame([(line[:len(line) //2], line[len(line) // 2:]) for line in input], columns=["left", "right"])
part1 = df.apply(lambda df:find_duplicate(df.left, df.right), axis=1).sum()

ser = pd.Series([input[i:i+3] for i in range(0, len(input) -2, 3)])
part2 = ser.apply(lambda x: find_duplicate2(x[0], x[1], x[2])).sum()

part1, part2


(8085, 2515)

# Day 4 - Camp Cleanup

In [3]:
import pandas as pd
import re

with open("./resources/day4.txt") as f:
    input = f.readlines()

pattern = re.compile(r"(\d+)-(\d+),(\d+)-(\d+)")

def is_contained(a_start: int, a_end: int, b_start: int, b_end:int) -> int:
    if (a_start<=b_start and a_end>=b_end) or (b_start<=a_start and b_end>=a_end):
        return 1 # a,b fully contains c,d or c,d fully contains a,b'
    return 0

def overlap(a_start: int, a_end: int, b_start: int, b_end:int) -> int:
    if a_start > b_start:
        return overlap(b_start, b_end, a_start, a_end)
    elif a_end>=b_start:
        return 1
    return 0


df = pd.DataFrame([re.match(pattern, line).groups() for line in input], columns=["a_start", "a_end", "b_start", "b_end"]).astype(int)
part1 = df.apply(lambda df: is_contained(df.a_start, df.a_end, df.b_start, df.b_end), axis=1).sum()
part2 = df.apply(lambda df: overlap(df.a_start, df.a_end, df.b_start, df.b_end), axis=1).sum()

part1, part2

'bcd'

# Day 5 - Supply Stacks

In [14]:
import re

with open("./resources/day5.txt") as f:
    input = f.readlines()

pattern = re.compile(r"move (\d+) from (\d+) to (\d+)")

def _gen_stacks() -> list:
    return [ "SLW","JTNQ","SCHFJ","TRMWNGB", "TRLSDHQB", "MJBVFHRL", "DWRNJM", "BZTFHNDJ", "HLQNBFT"]

def _get_result(stks: list) -> str:
    res = ""
    for stk in stks:
        res += stk[-1]
    return res

steps = [re.match(pattern, line).groups() for line in input]

def part1():
    stks = _gen_stacks()
    for step in steps:
        amount = int(step[0])
        for _ in range(amount):
            crate = stks[int(step[1]) -1][-1]
            stks[int(step[1]) -1] = stks[int(step[1]) -1].removesuffix(crate)
            stks[int(step[2]) -1] += crate
    return _get_result(stks)

def part2():
    stks = _gen_stacks()
    for step in steps:
        amount = int(step[0])
        crates = stks[int(step[1]) -1][-amount:]
        stks[int(step[1]) -1] = stks[int(step[1]) -1].removesuffix(crates)
        stks[int(step[2]) -1] += crates
    return _get_result(stks)

part1(), part2()

('RLFNRTNFB', 'MHQTLJRLB')

# Day 6 - Tuning Trouble

In [29]:
with open("./resources/day6.txt") as f:
    input = f.read()

def get_marker(input: str, packet_size: int):
    for i in range(len(input) - packet_size):
        current_pack = input[i:i+packet_size]
        if len(current_pack) == len(set(current_pack)):
            return i + packet_size
            
part1 = get_marker(input, 4)
part2 = get_marker(input, 14)

part1, part2

(1175, 3217)